In [1]:
%%time
import dask.dataframe as dd

import pandas as pd
import numpy as np
import time
# pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

# pd = dd

## TO DO: LOOK FOR MAX VALUES BETWEEN PASS THROW AND PASS CATCH INSTEAD OF THROUGH ALL

WEEKS = list(range(1, 6))
# WEEKS = [1, 4, 10]

og_df = pd.DataFrame()
# og_df = dd.read_csv('../data/week*_norm.csv', low_memory=False, usecols=['gameId', 'nflId', 'displayName', 'position', 'playId', 'frameId', 'v_mag', 'a_mag'])

#ingest
for week in WEEKS:
    og_df = og_df.append(pd.read_csv(f"../data/week{week}_norm.csv", ))
# data = pd.read_csv('./data/weektoy.csv', low_memory=False)
og_df = og_df.reset_index(drop=True)
og_df.tail()

CPU times: user 20.5 s, sys: 19.7 s, total: 40.2 s
Wall time: 40.1 s


,gameId,playId,frameId,event,nflId,displayName,jerseyNumber,position,position_general,team,team_pos,teamAbbr,route,time,los,x,y,dis,o,s,s_dir,s_dir_rad,v_x,v_y,v_theta,v_mag,a_old,a_x,a_y,a_theta,a_mag
5765388,2018100800,3855,66,None,2558019.0,Alvin Kamara,41.0,RB,RB,home,OFF,NO,FLAT,2018-10-09T03:09:13.500Z,40,39.65,7.55,0.14,66.48,1.38,70.04,1.22,1.30,0.47,0.35,1.38,1.56,-0.26,1.58,-1.41,1.60
5765389,2018100800,3855,66,None,2558133.0,Montae Nicholson,35.0,FS,S,away,DEF,WAS,NaN,2018-10-09T03:09:13.500Z,40,55.11,49.82,0.51,334.08,4.95,348.07,6.07,-1.02,4.84,-1.36,4.95,2.36,1.32,-2.06,-1.00,2.45
5765390,2018100800,3855,66,None,2558136.0,Fabian Moreau,31.0,CB,DB,away,DEF,WAS,NaN,2018-10-09T03:09:13.500Z,40,56.20,48.31,0.59,5.45,5.68,20.62,0.36,2.00,5.32,1.21,5.68,3.97,-1.71,-4.16,1.18,4.50
5765391,2018100800,3855,66,None,2560878.0,Tre'Quan Smith,10.0,WR,WR,home,OFF,NO,GO,2018-10-09T03:09:13.500Z,40,56.93,51.61,0.17,107.60,1.89,99.70,1.74,1.86,-0.32,-0.17,1.89,2.45,2.15,-0.47,-0.22,2.20
5765392,2018100800,3855,66,None,0.0,Football,NaN,NaN,NaN,football,FTBL,FTBL,NaN,2018-10-09T03:09:13.500Z,40,50.99,53.62,0.74,NaN,7.27,NaN,NaN,6.80,2.80,0.39,7.35,3.51,-1.00,-3.00,1.25,3.16


In [16]:
df = og_df.loc[og_df.nflId != 0]

In [17]:
%%time

df = pd.merge(df, df.groupby(['gameId', 'playId', 'nflId'])['v_mag'].rolling(20).mean().reset_index()[['level_3', 'v_mag']].set_index('level_3').rename(columns={'v_mag':'v_roll_20'}), left_index=True, right_index=True)

df = pd.merge(df, df.groupby(['gameId', 'playId', 'nflId'])['a_mag'].rolling(10).mean().reset_index()[['level_3', 'a_mag']].set_index('level_3').rename(columns={'a_mag':'a_roll_10'}), left_index=True, right_index=True)

df.tail()

CPU times: user 58.1 s, sys: 3.24 s, total: 1min 1s
Wall time: 1min 1s


,gameId,playId,frameId,event,nflId,displayName,jerseyNumber,position,position_general,team,team_pos,teamAbbr,route,time,los,x,y,dis,o,s,s_dir,s_dir_rad,v_x,v_y,v_theta,v_mag,a_old,a_x,a_y,a_theta,a_mag,v_roll_20,a_roll_10
5765387,2018100800,3855,66,None,2557853.0,Ryan Anderson,52.0,OLB,LB,away,DEF,WAS,NaN,2018-10-09T03:09:13.500Z,40,45.63,45.79,0.55,51.22,5.39,48.69,0.85,4.05,3.56,0.72,5.39,2.13,-1.00,-1.88,1.08,2.13,6.2200,1.575
5765388,2018100800,3855,66,None,2558019.0,Alvin Kamara,41.0,RB,RB,home,OFF,NO,FLAT,2018-10-09T03:09:13.500Z,40,39.65,7.55,0.14,66.48,1.38,70.04,1.22,1.30,0.47,0.35,1.38,1.56,-0.26,1.58,-1.41,1.60,2.8640,2.433
5765389,2018100800,3855,66,None,2558133.0,Montae Nicholson,35.0,FS,S,away,DEF,WAS,NaN,2018-10-09T03:09:13.500Z,40,55.11,49.82,0.51,334.08,4.95,348.07,6.07,-1.02,4.84,-1.36,4.95,2.36,1.32,-2.06,-1.00,2.45,6.7835,2.453
5765390,2018100800,3855,66,None,2558136.0,Fabian Moreau,31.0,CB,DB,away,DEF,WAS,NaN,2018-10-09T03:09:13.500Z,40,56.20,48.31,0.59,5.45,5.68,20.62,0.36,2.00,5.32,1.21,5.68,3.97,-1.71,-4.16,1.18,4.50,7.5555,2.407
5765391,2018100800,3855,66,None,2560878.0,Tre'Quan Smith,10.0,WR,WR,home,OFF,NO,GO,2018-10-09T03:09:13.500Z,40,56.93,51.61,0.17,107.60,1.89,99.70,1.74,1.86,-0.32,-0.17,1.89,2.45,2.15,-0.47,-0.22,2.20,2.4475,4.947


In [24]:
%%time
play_maxes = df.groupby(['gameId', 'playId', 'nflId', 'displayName', 'position', 'position_general'], as_index=False)['v_roll_20'].max()[['nflId', 'displayName','position', 'position_general', 'v_roll_20']]
play_maxes.tail()

CPU times: user 1.63 s, sys: 488 ms, total: 2.12 s
Wall time: 2.12 s


,nflId,displayName,position,position_general,v_roll_20
82694,2557853.0,Ryan Anderson,OLB,LB,6.2410
82695,2558019.0,Alvin Kamara,RB,RB,6.7835
82696,2558133.0,Montae Nicholson,FS,S,7.1260
82697,2558136.0,Fabian Moreau,CB,DB,8.3745
82698,2560878.0,Tre'Quan Smith,WR,WR,7.0415


In [25]:
player_maxes = play_maxes.groupby(['nflId', 'displayName', 'position', 'position_general'],  as_index=False).max().dropna().set_index(['nflId'])

In [26]:
def rename(newname):
    def decorator(f):
        f.__name__ = newname
        return f
    return decorator

def q_at(y):
    @rename(f'Q{y:0.2f}')
    def q(x):
        return x.quantile(y)
    return q

stats_cat = player_maxes[['displayName', 'position', 'position_general', 'v_roll_20']].groupby('position_general').agg([q_at(.5), q_at(.6), q_at(.7), q_at(.8), q_at(.9), q_at(.95), q_at(.99)]).round(2)
stats_cat['player_count'] = player_maxes.groupby('position_general')['displayName'].count()

stats_pos = player_maxes[['displayName', 'position', 'position_general', 'v_roll_20']].groupby('position').agg([q_at(.5), q_at(.6), q_at(.7), q_at(.8), q_at(.9), q_at(.95), q_at(.99)]).round(2)
stats_pos['player_count'] = player_maxes.groupby('position')['displayName'].count()

In [27]:
stats_cat.sort_values(by=('v_roll_20', 'Q0.60'), ascending=False).style.background_gradient(cmap='viridis').format("{:4.2f}")

In [28]:
stats_pos.sort_values(by=('v_roll_20', 'Q0.60'), ascending=False).style.background_gradient(cmap='viridis').format("{:4.2f}")

In [30]:
player_maxes.loc[player_maxes.position=='TE'].sort_values('v_roll_20', ascending=False).reset_index(drop=True).head(50).style.background_gradient(cmap='viridis')

,displayName,position,position_general,v_roll_20
0,O.J. Howard,TE,TE,9.799500
1,Evan Engram,TE,TE,9.677000
2,George Kittle,TE,TE,9.514500
3,Vernon Davis,TE,TE,9.393500
4,Gerald Everett,TE,TE,9.369500
5,Jordan Reed,TE,TE,9.276000
6,Jesse James,TE,TE,9.252000
7,Tyler Higbee,TE,TE,9.195000
8,Jared Cook,TE,TE,9.162000
9,Rico Gathers,TE,TE,9.141500


In [31]:
df['v_mag_bin'] = pd.cut(df.v_mag, bins=[-1, 6, 9, 12])
df.head()

,gameId,playId,frameId,event,nflId,displayName,jerseyNumber,position,position_general,team,team_pos,teamAbbr,route,time,los,x,y,dis,o,s,s_dir,s_dir_rad,v_x,v_y,v_theta,v_mag,a_old,a_x,a_y,a_theta,a_mag,v_roll_20,a_roll_10,v_mag_bin
0,2018090600,75,1,None,310.0,Matt Ryan,2.0,QB,QB,away,OFF,ATL,NaN,2018-09-07T01:07:14.599Z,40,38.38,26.63,0.02,109.57,0.00,60.93,1.06,0.00,0.00,0.00,0.00,0.01,0.0,0.0,0.0,0.0,NaN,NaN,"(-1, 6]"
1,2018090600,75,1,None,79848.0,Malcolm Jenkins,27.0,SS,S,home,DEF,PHI,NaN,2018-09-07T01:07:14.599Z,40,41.22,16.83,0.01,285.63,0.01,246.66,4.31,-0.01,-0.00,0.41,0.01,0.01,0.0,0.0,0.0,0.0,NaN,NaN,"(-1, 6]"
2,2018090600,75,1,None,2495454.0,Julio Jones,11.0,WR,WR,away,OFF,ATL,HITCH,2018-09-07T01:07:14.599Z,40,38.76,9.14,0.01,110.45,0.02,196.86,3.44,-0.01,-0.02,1.28,0.02,0.03,0.0,0.0,0.0,0.0,NaN,NaN,"(-1, 6]"
3,2018090600,75,1,None,2495613.0,Corey Graham,24.0,FS,S,home,DEF,PHI,NaN,2018-09-07T01:07:14.599Z,40,43.80,31.29,0.01,250.12,0.09,348.91,6.09,-0.02,0.09,-1.38,0.09,0.42,0.0,0.0,0.0,0.0,NaN,NaN,"(-1, 6]"
4,2018090600,75,1,None,2533040.0,Mohamed Sanu,12.0,WR,WR,away,OFF,ATL,HITCH,2018-09-07T01:07:14.599Z,40,39.33,17.15,0.00,77.61,0.00,13.97,0.24,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,NaN,NaN,"(-1, 6]"


In [33]:
player_maxes = df[['displayName', 'position', 'position_general', 'v_mag_bin', 'a_roll_10']].groupby(['displayName', 'v_mag_bin'], as_index=False).max()
player_maxes.loc[player_maxes.displayName=='Tyreek Hill']

,displayName,v_mag_bin,position,position_general,a_roll_10
2856,Tyreek Hill,"(-1, 6]",WR,WR,14.284
2857,Tyreek Hill,"(6, 9]",WR,WR,8.266
2858,Tyreek Hill,"(9, 12]",WR,WR,5.610


In [35]:
stats_cat = player_maxes.groupby(['position_general', 'v_mag_bin'], as_index=False).agg([q_at(.5), q_at(.6), q_at(.7), q_at(.8), q_at(.9), q_at(.95), q_at(.99)]).round(2)

stats_pos = player_maxes.groupby(['position', 'v_mag_bin'], as_index=False).agg([q_at(.5), q_at(.6), q_at(.7), q_at(.8), q_at(.9), q_at(.95), q_at(.99)]).round(2) 

In [36]:
stats_cat.loc[['WR', 'DB', 'S', 'RB', 'TE', 'LB', 'FB', 'QB', 'DL', 'ST'], :].dropna().style.background_gradient(cmap='viridis').format("{:4.2f}")

In [37]:
stats_cat.sort_values(by=('a_roll_10', 'Q0.60'), ascending=False).style.background_gradient(cmap='viridis').format("{:4.2f}")

In [39]:
player_maxes.loc[player_maxes.position=='TE'].sort_values('a_roll_10', ascending=False).reset_index(drop=True).head(50).style.background_gradient(cmap='viridis')

,displayName,v_mag_bin,position,position_general,a_roll_10
0,Dallas Goedert,"(9, 12]",TE,TE,14.073000
1,Dallas Goedert,"(6, 9]",TE,TE,13.938000
2,Zach Ertz,"(6, 9]",TE,TE,9.982000
3,Zach Ertz,"(-1, 6]",TE,TE,9.876000
4,Dallas Goedert,"(-1, 6]",TE,TE,9.118000
5,Mike Gesicki,"(-1, 6]",TE,TE,8.504000
6,Jimmy Graham,"(-1, 6]",TE,TE,8.477000
7,Luke Stocker,"(-1, 6]",TE,TE,8.476000
8,Austin Hooper,"(6, 9]",TE,TE,8.461000
9,Zach Ertz,"(9, 12]",TE,TE,8.458000


In [17]:
# BELOW THIS IS EXPERIMENTAL STUFF IM KEEPIGN FOR REFERENCE

In [18]:
motion = track_df[['gameId', 'playId', 'position', 's', 'a']]
maxes = motion.groupby(['gameId', 'playId', 'position'], as_index=False).max()
maxes.head()


NameError: name 'track_df' is not defined

In [19]:
stats = maxes.drop(['gameId', 'playId'], axis=1).groupby('position').describe(percentiles=[0.5, 0.8, 0.9, 0.95, 0.99])
stats

NameError: name 'maxes' is not defined

In [14]:
stats.drop([('s', 'std'), ('s', 'min'), ('s', 'max'), ('s', 'mean'), ('a', 'std'), ('a', 'min'), ('a', 'count'), ('a', 'max'), ('a', 'mean')], axis=1).sort_values(by=('a', '95%'), ascending=False).round(2)

s                                    a                    \
            count   50%   80%   90%   95%    99%   50%   80%   90%   95%   
position                                                                   
WR        19147.0  7.83  8.74  9.16  9.49  10.03  6.17  7.27  8.25  9.68   
TE        18693.0  6.33  7.32  7.82  8.19   8.82  5.02  6.08  6.80  7.59   
CB        19197.0  6.99  8.24  8.83  9.28  10.00  5.64  6.60  7.12  7.59   
OLB       15863.0  5.66  6.77  7.42  7.92   8.83  5.07  6.15  6.83  7.57   
FB          632.0  6.10  7.04  7.52  8.01   8.81  4.88  5.98  6.72  7.32   
RB        18267.0  5.65  6.94  7.56  8.13   9.09  4.66  5.77  6.44  7.16   
DL           37.0  5.12  6.01  6.52  6.99   7.39  4.98  5.63  6.18  7.09   
HB          632.0  5.82  6.87  7.58  8.10   8.80  4.83  5.89  6.60  7.05   
ILB        9440.0  5.92  7.10  7.67  8.12   8.86  4.92  5.94  6.49  7.04   
LB         7153.0  5.74  6.98  7.59  8.12   8.90  4.87  5.84  6.37  6.92   
DB         5136.0  5.94  7.30  8.04  8.61   9.48  4.69  5.80  6.38  6.88   
MLB        6227.0  5.62  6.84  7.48  8.01   8.91  4.67  5.69  6.24  6.76   
FS        13909.0  6.14  7.34  7.98  8.52   9.36  4.80  5.77  6.30  6.75   
SS        12762.0  5.94  7.20  7.83  8.36   9.25  4.72  5.75  6.27  6.70   
S          1680.0  5.85  7.11  7.84  8.40   9.26  4.56  5.60  6.13  6.60   
QB        19199.0  3.05  4.62  5.59  6.49   7.88  3.54  4.68  5.49  6.23   
DE          139.0  5.29  6.27  6.78  7.44   8.19  4.22  5.22  5.54  6.23   
P            16.0  4.28  6.36  6.66  6.96   7.11  3.00  4.24  4.90  5.92   
DT           23.0  4.85  6.73  6.85  6.99   7.55  3.85  4.88  5.65  5.83   
NT            5.0  4.04  5.12  5.79  6.12   6.38  3.48  3.84  4.08  4.20   
LS           16.0  4.64  6.06  6.35  7.24   8.82  2.84  3.54  3.58  3.78   
K             5.0  4.77  5.89  6.40  6.66   6.86  2.90  3.48  3.55  3.59   

                 
            99%  
position         
WR        12.04  
TE         9.82  
CB         8.83  
OLB        9.43  
FB         9.77  
RB         9.33  
DL         8.38  
HB         9.11  
ILB        8.46  
LB         8.42  
DB         7.79  
MLB        8.16  
FS         7.74  
SS         7.75  
S          7.55  
QB         8.02  
DE         7.59  
P          8.02  
DT         6.38  
NT         4.30  
LS         4.20  
K          3.61

In [ ]:
motion = track_df[['gameId', 'playId', 'nflId', 'displayName','position', 's', 'a']]
maxes = motion2.groupby(['nflId', 'gameId', 'playId', 'displayName', 'position'], as_index=False).agg(
    frames = ('s', 'count'),
    max_s = ('s', 'max'),
    max_10th_s = ('s', lambda x: x.nlargest(10).iloc[-1]),
    max_15th_s = ('s', lambda x: x.nlargest(15).iloc[-1]),
    max_20th_s = ('s', lambda x: x.nlargest(20).iloc[-1]),
    max_a = ('a', 'max'),
    max_10th_a = ('a', lambda x: x.nlargest(10).iloc[-1]),
    max_15th_a = ('a', lambda x: x.nlargest(15).iloc[-1]),
    max_20th_a = ('a', lambda x: x.nlargest(20).iloc[-1]),
)

In [24]:
weeks = [15, 16]
toy = pd.DataFrame()
for week in weeks:
    toy = toy.append(pd.read_csv(f"../data/week{week}_norm.csv", low_memory=False))

In [56]:
df = toy.loc[toy.team != 'football']

df['roll_s_30'] = df.groupby(['gameId', 'playId', 'nflId'])['s'].transform(lambda x: x.rolling(30).mean())
df['roll_s_20'] = df.groupby(['gameId', 'playId', 'nflId'])['s'].transform(lambda x: x.rolling(20).mean())
df['roll_s_10'] = df.groupby(['gameId', 'playId', 'nflId'])['s'].transform(lambda x: x.rolling(10).mean())
df['roll_a_5'] = df.groupby(['gameId', 'playId', 'nflId'])['a'].transform(lambda x: x.rolling(5).mean())
df['roll_a_10'] = df.groupby(['gameId', 'playId', 'nflId'])['a'].transform(lambda x: x.rolling(5).mean())

In [61]:
motion = df[['gameId', 'playId', 'nflId', 'displayName','position', 's', 'roll_s_30', 'roll_s_20', 'roll_s_10','a', 'roll_a_5']]
maxes = motion.groupby(['nflId', 'displayName', 'position']).max()

# .agg(
#     frames = ('s', 'count'),
#     max_s = ('s', 'max'),
#     max_10th_s = ('s', lambda x: x.nlargest(10).iloc[-1]),
#     max_20th_s = ('s', lambda x: x.nlargest(20).iloc[-1]),
#     max_50th_s = ('s', lambda x: x.nlargest(50).iloc[-1]),
#     max_100th_s = ('s', lambda x: x.nlargest(100).iloc[-1]),
#     max_a = ('a', 'max'),
#     max_10th_a = ('a', lambda x: x.nlargest(10).iloc[-1]),
#     max_20th_a = ('a', lambda x: x.nlargest(20).iloc[-1]),
#     max_50th_a = ('a', lambda x: x.nlargest(50).iloc[-1]),
#     max_100th_a = ('a', lambda x: x.nlargest(100).iloc[-1]),
# )
# maxes.sort_values('max_s', ascending=False).head(20)

In [70]:
maxes['diff'] = maxes['roll_s_10']-maxes['roll_s_30']
maxes.sort_values('roll_s_20', ascending=False).head(100)
# maxes['diff'].describe()

,,,gameId,playId,s,roll_s_30,roll_s_20,roll_s_10,a,roll_a_5,diff
nflId,displayName,position,,,,,,,,,
2560725.0,Minkah Fitzpatrick,S,2018122302,3610,10.68,10.364000,10.4880,10.602,8.93,8.234,-0.238000
2552608.0,Stefon Diggs,WR,2018122307,3747,10.51,10.192000,10.3655,10.443,6.73,6.400,-0.251000
2543474.0,Darqueze Dennard,CB,2018122305,4235,10.35,10.214667,10.2425,10.248,7.18,6.736,-0.033333
2534436.0,Deonte Thompson,WR,2018122308,3130,10.53,10.132333,10.2295,10.338,8.79,7.242,-0.205667
2507166.0,Ted Ginn,WR,2018122313,4285,10.54,9.964333,10.2210,10.393,11.71,9.922,-0.428667
2558010.0,John Ross,WR,2018122305,4430,10.38,9.975667,10.1905,10.308,12.59,11.194,-0.332333
2543501.0,Aaron Colvin,CB,2018122310,4645,10.33,10.052333,10.1680,10.257,7.80,7.442,-0.204667
2543468.0,Mike Evans,WR,2018122306,4098,10.28,10.059667,10.1195,10.194,11.78,10.524,-0.134333
2559053.0,Torry McTyer,CB,2018122302,3610,10.35,9.712667,10.1005,10.228,6.67,6.366,-0.515333


In [3]:
pd.set_option('display.max_colwidth', 200)
# play_events = track_df[['gameId', 'playId', 'frameId', 'event']].groupby(['gameId', 'playId', 'frameId'], as_index=False).first().sort_values(by=['gameId', 'playId', 'frameId']).groupby(['gameId', 'playId']).agg({'frameId': 'nunique', 'event': lambda x: list(filter(lambda y: y!='None', x))})
play_events = track_df[['gameId', 'playId', 'frameId', 'event']].groupby(['gameId', 'playId', 'frameId'], as_index=False).first().sort_values(by=['gameId', 'playId', 'frameId']).groupby(['gameId', 'playId']).agg(frames = ('frameId', 'nunique'), events = ('event', lambda x: list(filter(lambda y: y!='None', x))))
play_events.head(20)

frames                                             events
gameId     playId                                                           
2018122200 88          80  [ball_snap, play_action, pass_forward, pass_ou...
           110         78  [ball_snap, pass_forward, pass_arrived, pass_o...
           163         43  [ball_snap, pass_forward, pass_outcome_incompl...
           218         83  [ball_snap, pass_forward, pass_arrived, pass_o...
           289         62  [ball_snap, play_action, pass_forward, pass_ar...
           358         77  [ball_snap, pass_forward, pass_arrived, pass_o...
           383         47  [ball_snap, pass_forward, pass_arrived, pass_o...
           478         66  [man_in_motion, ball_snap, pass_shovel, pass_a...
           787         95  [ball_snap, play_action, pass_forward, pass_ar...
           834         65  [ball_snap, pass_forward, pass_arrived, pass_o...
           904         52  [shift, ball_snap, pass_forward, pass_tipped, ...
           995         84  [ball_snap, play_action, pass_forward, pass_ar...
           1043        71                     [line_set, ball_snap, qb_sack]
           1085        99  [ball_snap, pass_forward, pass_arrived, pass_o...
           1176        63  [ball_snap, pass_forward, pass_arrived, pass_o...
           1322        74  [man_in_motion, ball_snap, play_action, pass_f...
           1353        67  [ball_snap, pass_forward, pass_arrived, pass_o...
           1400        64  [man_in_motion, ball_snap, play_action, pass_f...
           1479        51                  [ball_snap, play_action, qb_sack]
           1499        85  [shift, ball_snap, pass_forward, pass_arrived,...

In [4]:
ends = track_df.loc[(track_df.event=='pass_forward') | (track_df.event=='qb_sack') | (track_df.event=='pass_shovel')].groupby(['gameId', 'playId', 'frameId'], as_index=False).first()[['gameId', 'playId', 'frameId']].rename({'frameId': 'prePassEndFrame'}, axis=1).dropna(axis=0).set_index(['gameId', 'playId'])

track_df = track_df.join(ends, on=['gameId','playId'])

track_df.groupby(['gameId', 'playId']).first()[['frameId', 'event', 'displayName', 'prePassEndFrame']]

frameId      event       displayName  prePassEndFrame
gameId     playId                                                       
2018122200 88            1       None      Josh Johnson             53.0
           110           1       None      Josh Johnson             39.0
           163           1       None      Josh Johnson             27.0
           218           1       None      Josh Johnson             71.0
           289           1       None     Ryan Kerrigan             33.0
...                    ...        ...               ...              ...
2018122400 3900          1       None      Jordy Nelson             30.0
           3978          1       None  Marcus Gilchrist             26.0
           4003          1  ball_snap  Marcus Gilchrist             29.0
           4073          1       None  Marcus Gilchrist             36.0
           4129          1       None  Marcus Gilchrist             38.0

[1210 rows x 4 columns]

In [5]:
track_df = track_df.loc[(track_df.frameId >= 10) & (track_df.frameId <= track_df.prePassEndFrame)]
# track_df = track_df.loc[track_df.team_pos!='FTBL']

In [6]:
plays = list(track_df.groupby(['gameId', 'playId'], as_index=False).first()[['gameId', 'playId']].to_records(index=False))

In [7]:
# Select the game and play that you wish to see in week 1
import random

game_id = 2018122314
play_id = 4239

# game_id, play_id = random.choice(plays)

unique_game_ids = track_df.gameId.unique()
unique_play_ids = track_df[track_df.gameId == game_id].playId.unique()

play_df = track_df[track_df.playId == play_id].sort_values(by = 'time')
play_df.head()

,gameId,playId,frameId,event,nflId,displayName,jerseyNumber,position,team,team_pos,teamAbbr,route,time,los,x,y,s,dis,a,o,dir,dir_rad,v_x,v_y,v_theta,prePassEndFrame
1073926,2018122314,4239,10,None,497224.0,Ed Dickson,84.0,TE,home,OFF,SEA,GO,2018-12-24T04:26:43.400Z,40,38.37,19.16,0.00,0.00,0.00,80.57,268.01,4.677657,-0.000000,-0.000000,0.000000,39.0
1073939,2018122314,4239,10,None,NaN,Football,NaN,NaN,football,FTBL,FTBL,NaN,2018-12-24T04:26:43.400Z,40,39.52,23.70,0.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN,0.000000,39.0
1073938,2018122314,4239,10,None,2561417.0,Charvarius Ward,35.0,DB,away,DEF,KC,NaN,2018-12-24T04:26:43.400Z,40,39.89,10.68,0.14,0.01,0.43,269.87,191.17,3.336546,-0.027121,-0.137348,1.375843,39.0
1073937,2018122314,4239,10,None,2561103.0,Tremon Smith,39.0,DB,away,DEF,KC,NaN,2018-12-24T04:26:43.400Z,40,40.61,33.44,0.02,0.00,0.40,266.10,161.25,2.814343,0.006429,-0.018939,-1.243547,39.0
1073936,2018122314,4239,10,None,2558865.0,Chris Carson,32.0,RB,home,OFF,SEA,NaN,2018-12-24T04:26:43.400Z,40,34.59,26.27,0.00,0.00,0.00,106.43,148.15,2.585705,0.000000,-0.000000,0.000000,39.0


In [8]:
motion = track_df[['dis', 's', 'v_x', 'v_y', 'a']]
motion.describe()

,dis,s,v_x,v_y,a
count,536076.000000,536076.000000,499504.000000,499504.000000,536076.000000
mean,0.305874,3.071834,1.363173,-0.129083,2.219274
std,0.234060,2.278834,2.661777,2.317083,1.585410
min,0.000000,0.000000,-8.209913,-10.105628,0.000000
25%,0.110000,1.140000,-0.087530,-1.088854,1.040000
50%,0.270000,2.790000,0.924624,0.000000,2.010000
75%,0.460000,4.690000,3.062776,0.845777,3.150000
max,5.540000,27.050000,9.697260,9.292045,25.470000


In [7]:
track_df.loc[track_df.position=="OLB"].displayName.unique()

array(['Derrick Morgan', 'Harold Landry', 'Ryan Kerrigan',
       'Preston Smith', 'Pernell McPhee', 'Ryan Anderson',
       'Terrell Suggs', 'Matt Judon', 'Jatavis Brown', "Za'Darius Smith",
       'Kyle Emanuel', 'Tyus Bowser', 'Olivier Vernon', 'Clay Matthews',
       'Kyler Fackrell', 'Nigel Bradham', 'Kamu Grugier-Hill',
       'Vontaze Burfict', 'Nick Vigil', 'Kareem Martin', 'Lorenzo Carter',
       'Telvin Smith', 'Myles Jack', 'Kyle Van Noy', 'Anthony Barr',
       'Eric Wilson', 'Vic Beasley', "De'Vondre Campbell",
       'Lorenzo Alexander', 'Whitney Mercilus', 'Neville Hewitt',
       'Jordan Jenkins', 'Devon Kennard', 'Eli Harold', 'Romeo Okwara',
       'Jamie Collins', 'Ben Gedeon', 'Thomas Davis', 'Lavonte David',
       'Devante Bond', 'Damien Wilson', 'Connor Barwin', 'Vincent Rey',
       'Albert McClellan', 'Leonard Floyd', 'Samson Ebukam', 'A.J. Klein',
       'Alex Anzalone', 'Bud Dupree', 'T.J. Watt', 'Craig Robertson',
       'Anthony Chickillo', 'Matt Longacre'

In [8]:
track_df.loc[track_df.position=="ILB"].displayName.unique()

array(['Wesley Woodyard', 'Jayon Brown', 'Mason Foster', 'Zach Brown',
       'Kamalei Correa', 'Daren Bates', 'C.J. Mosley', 'Patrick Onwuasor',
       'B.J. Goodson', 'Tae Davis', 'Antonio Morrison', 'Blake Martinez',
       'Ukeme Eligwe', 'Benardrick McKinney', 'Zach Cunningham',
       'Ahmad Thomas', 'Matthew Adams', 'Zaire Franklin',
       'Avery Williamson', 'Dylan Cole', 'Christian Jones',
       'Deone Bucannon', 'Haason Reddick', 'Danny Trevathan',
       'Mark Barron', 'Cory Littleton', 'Demario Davis', 'L.J. Fort',
       'Vince Williams', 'Jon Bostic', 'Bryce Hager', 'Ramik Wilson',
       'Reggie Ragland', 'Nicholas Morrow', 'Todd Davis',
       'Brandon Marshall'], dtype=object)

In [9]:
track_df.loc[track_df.position=="MLB"].displayName.unique()

array(['Hayes Pullard', 'Jaylon Smith', 'Jordan Hicks', 'Hardy Nickerson',
       'Deion Jones', "Dont'a Hightower", 'Elandon Roberts',
       'Jarrad Davis', 'Mike Hull', 'Joe Schobert', 'Luke Kuechly',
       'Adarius Taylor', 'Kentrell Brothers', 'Mark Nzeocha',
       'Bobby Wagner', 'Anthony Hitchens', 'Marquel Lee'], dtype=object)